In [1]:
nn = require("nn")
nninit = require("nninit")
require 'optim'
csvigo =  require("csvigo")
require 'torch'

Read data

In [2]:
-- load trainning_size
matrix_words = csvigo.load{path='pre-processing/train_data/good_train_matrix_file.txt', mode='large', separator=' '}
classes = {"O", "B-LOC", "B-PER", "B-ORG", "B-TOUR", "I-ORG", "I-PER", "I-TOUR", "I-LOC", "B-PRO", "I-PRO"}

windowSize = #matrix_words[1]
trainingSize = #matrix_words

data = torch.Tensor(trainingSize, windowSize)
for i=1, trainingSize do
    for j=1, windowSize do
        data[i][j] = tonumber(matrix_words[i][j]) + 1 -- lua index start from 1, not 0
    end
end


-- read labels
labelsRaw = csvigo.load{path='pre-processing/train_data/good_train_label_file.txt', mode='large', separator=' '}
labels = torch.DoubleTensor(#labelsRaw)
for i=1, #labelsRaw do    
    labels[i] = tonumber(labelsRaw[i][1]) + 1
end

print(string.format("window_size = %d, training_size = %d", windowSize, trainingSize))

<csv>	parsing file: pre-processing/train_data/good_train_matrix_file.txt	


<csv>	parsing done	


<csv>	parsing file: pre-processing/train_data/good_train_label_file.txt	


<csv>	parsing done	


window_size = 11, training_size = 11456	


In [3]:
-- load test_data -- 
-- load trainning_size
matrix_words_test = csvigo.load{path='pre-processing/train_data/good_test_matrix_file.txt', mode='large', separator=' '}
testingSize = #matrix_words_test

data_test = torch.Tensor(testingSize, windowSize)
for i=1, testingSize do
    for j=1, windowSize do        
        data_test[i][j] = tonumber(matrix_words_test[i][j]) + 1 -- lua index start from 1, not 0
    end
end


-- read labels -- 
labelsRawTest = csvigo.load{path='pre-processing/train_data/good_test_label_file.txt', mode='large', separator=' '}
labels_test = torch.DoubleTensor(#labelsRawTest)
for i=1, #labelsRawTest do    
     labels_test[i] = tonumber(labelsRawTest[i][1]) + 1
end

print(string.format("window_size = %d, testing_size = %d", windowSize, testingSize))

<csv>	parsing file: pre-processing/train_data/good_test_matrix_file.txt	


<csv>	parsing done	


<csv>	parsing file: pre-processing/train_data/good_test_label_file.txt	


<csv>	parsing done	


window_size = 11, testing_size = 333446	


Config some parameters

In [4]:
-- load data 
w2v_mat = csvigo.load{path='pre-processing/train_data/total_matrix_final.txt', mode='large', separator=' '}

rows = #w2v_mat
cols = #w2v_mat[1] - 1 -- the last elem is \n

-- load word
words = csvigo.load{path='pre-processing/train_data/total_words_final.txt', mode='large', separator=' '}
dictSize = #words
embeddedSize = cols

print (string.format("Size {rows=%d, cols=%d}", rows, cols))
print (string.format("Number of total words = %d", dictSize))

<csv>	parsing file: pre-processing/train_data/total_matrix_final.txt	


<csv>	parsing done	


<csv>	parsing file: pre-processing/train_data/total_words_final.txt	


<csv>	parsing done	


Size {rows=16458, cols=200}	
Number of total words = 16458	


In [5]:
csv_tensor = torch.Tensor(rows, cols)
print(string.format("Generate new tensor at size %d x %d", rows, cols))

Generate new tensor at size 16458 x 200	


Load csv data to a Tensor

In [6]:
for i=1, rows do
    for j=1, cols do                 
        csv_tensor[i][j] = tonumber(w2v_mat[i][j])
    end
end


Create lookup table

In [7]:
lookup = nn.LookupTable(dictSize, embeddedSize)
lookup.weight = csv_tensor

print(#lookup:forward(data[1]))

  11
 200
[torch.LongStorage of size 2]



Create Model

In [8]:
-- some matrix layer --
L = 20
K = embeddedSize

windowSize = 11
model = nn.Sequential()
model:add(lookup)
model:add(nn.Reshape(K*windowSize))
model:add(nn.Linear(K*windowSize, L))
model:add(nn.HardTanh())
model:add(nn.Linear(L, #classes))
model:add(nn.SoftMax())

print(model)


nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> output]
  (1): nn.LookupTable
  (2): nn.Reshape(2200)
  (3): nn.Linear(2200 -> 20)
  (4): nn.HardTanh
  (5): nn.Linear(20 -> 11)
  (6): nn.SoftMax
}
{
  gradInput : DoubleTensor - empty
  modules : 
    {
      1 : 
        nn.LookupTable
        {
          output : DoubleTensor - size: 11x200
          gradInput : DoubleTensor - empty
          copiedInput : true
          weight : DoubleTensor - size: 16458x200
          _type : torch.DoubleTensor
          _input : LongTensor - size: 11
          paddingValue : 0
          _count : IntTensor - empty
          shouldScaleGradByFreq : false
          gradWeight : DoubleTensor - size: 16458x200
        }
      2 : 
        nn.Reshape(2200)
        {
          _type : torch.DoubleTensor
          output : DoubleTensor - empty
          gradInput : DoubleTensor - empty
          size : LongStorage - size: 1
          nelement : 2200
          batchsize : LongStorage -

In [9]:
criterion = nn.ClassNLLCriterion()
x, dl_dx = model:getParameters()

In [10]:
sgd_params = {
   learningRate = 1e-2,
   learningRateDecay = 1e-4,
   weightDecay = 1e-3,
   momentum = 1e-4
}

In [11]:
print(trainingSize)
step = function(batch_size)
    local current_loss = 0
    local count = 0
    local shuffle = torch.randperm(trainingSize)    
    batch_size = batch_size or 200
    
    for t = 0, trainingSize, batch_size do
        if t==trainingSize then
            break
        end
        
        -- setup inputs and targets for this mini-batch
        local size = math.min(t + batch_size, trainingSize) - t        
        local inputs = torch.Tensor(size, windowSize)
        local targets = torch.Tensor(size)
        for i = 1, size do            
            local input = data[shuffle[i+t]]
            local target = labels[shuffle[i+t]]            
            -- if target == 0 then target = 10 end
            inputs[i] = input
            targets[i] = target
        end
        
        local feval = function(x_new)
            -- reset data
            if x ~= x_new then x:copy(x_new) end
            dl_dx:zero()

            -- perform mini-batch gradient descent            
            outputs = model:forward(inputs);            
            local loss = criterion:forward(model:forward(inputs), targets)
            model:backward(inputs, criterion:backward(model.output, targets))

            return loss, dl_dx
        end
        
        _, fs = optim.sgd(feval, x, sgd_params)
        -- fs is a table containing value of the loss function
        -- (just 1 value for the SGD optimization)
        count = count + 1
        current_loss = current_loss + fs[1]        
   end

    -- normalize loss
    return current_loss / count
end

11456	


In [12]:
-- eval --
print(testingSize)
eval = function(batch_size)
    local count = 0
    batch_size = batch_size or 200        
    
    for i = 1, testingSize, batch_size do
        local size = math.min(i + batch_size - 1, testingSize) - i
        local inputs = data_test[{{i,i+size-1}}]
        local targets = labels_test[{{i,i+size-1}}]:long()
        local outputs = model:forward(inputs)    
        local _, indices = torch.max(outputs, 2)                
        local guessed_right = indices:eq(targets):sum()
        
        count = count + guessed_right
    end

    return count / testingSize
end

333446	


In [ ]:
max_iters = 30
do
    local last_accuracy = 0
    local decreasing = 0
    local threshold = 1 -- how many deacreasing epochs we allow
    for i = 1, max_iters do
        local loss = step(50)        
        print(string.format('Epoch: %d Current loss: %4f', i, loss))
        local accuracy = eval(50)
        print(string.format('Accuracy on the validation set: %4f', accuracy))
        if accuracy < last_accuracy then
             if decreasing > threshold then break end
             decreasing = decreasing + 1
        else
             decreasing = 0
        end
        last_accuracy = accuracy
    end
end

Epoch: 1 Current loss: -0.099047	
